In \[2\]:

    unsw_rdd = sc.textFile("/home/wednesday_group/Downloads/UNSW-NB15.csv")

In \[3\]:

    unsw_rdd.take(2)

Out\[3\]:

    ['59.166.0.3,56716,149.171.126.8,143,tcp,FIN,0.82546002,7812,16236,31,29,30,32,-,75090.25,156111.73,122,126,255,255,2751097753,2748686736,64,129,0,0,445.25928,474.9451,1421970774,1421970775,6.8190908,6.599896,5.9700001E-4,4.6899999E-4,0.000128,0,0,0,0,0,2,7,1,4,1,1,1,,0',
     '59.166.0.0,43467,149.171.126.6,49729,tcp,FIN,0.101815,4238,65628,31,29,7,30,-,328438.84,5087030.5,72,74,255,255,961515433,3225510659,59,887,0,0,0,91.579567,1421970775,1421970775,1.429493,1.387192,0.00068,5.4600002E-4,0.000134,0,0,0,0,0,7,4,1,6,1,1,1,,0']

In \[4\]:

    from pyspark.mllib.regression import LabeledPoint
    from numpy import array

    def parse_interaction(line):
        line_split = line.split(",")
        # leave_out = [1,2,3,41]
        clean_line_split = [6,7,8,9,10,11,12,16,17,18,19,20,21,22,23,24,25,26,27,30,31,32,33,34,36,43,44,45,46,40,41,42,48,35]
        attack = 1.0
        if line_split[47]== '':
            attack = 0.0
        return LabeledPoint(attack, array([float(x) for x in clean_line_split]))

    main_data = unsw_rdd.map(parse_interaction)

In \[5\]:

    main_data.take(2)

Out\[5\]:

    [LabeledPoint(0.0, [6.0,7.0,8.0,9.0,10.0,11.0,12.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,43.0,44.0,45.0,46.0,40.0,41.0,42.0,48.0,35.0]),
     LabeledPoint(0.0, [6.0,7.0,8.0,9.0,10.0,11.0,12.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,30.0,31.0,32.0,33.0,34.0,36.0,43.0,44.0,45.0,46.0,40.0,41.0,42.0,48.0,35.0])]

In \[6\]:

    [train_data, test_data] = main_data.randomSplit([0.7, 0.3], seed = 1234)

In \[7\]:

    #[train_data.count(), test_data.count()]

In \[8\]:

    from pyspark.mllib.classification import LogisticRegressionWithLBFGS
    from time import time

    # Build the model
    t0 = time()
    logit_model = LogisticRegressionWithLBFGS.train(train_data)
    tt = time() - t0 # to know how long it takes to complete Regression analysis

    print ("Classifier trained in {} seconds".format(round(tt,3)))

    Classifier trained in 100.681 seconds

In \[9\]:

    labels_and_preds_train = train_data.map(lambda a: (abs(a.label)-abs(logit_model.predict(a.features))))

In \[10\]:

    labels_and_preds_train.take(10)

Out\[10\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[11\]:

    # Model accuracy on training data
    t0 = time()
    train_accuracy = labels_and_preds_train.filter(lambda a: a==0).count() / float(train_data.count())
    tt = time() - t0
    print ("Prediction made in {} seconds. Training accuracy is {}".format(round(tt,3), round(train_accuracy,4)))

    Prediction made in 100.285 seconds. Training accuracy is 0.8736

In \[12\]:

    labels_and_preds_test = test_data.map(lambda a: (abs(a.label)-abs(logit_model.predict(a.features))))

In \[13\]:

    labels_and_preds_test.take(10)

Out\[13\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[14\]:

    # model accuracy for test data
    t0 = time()
    test_accuracy = labels_and_preds_test.filter(lambda a: a==0).count() / float(test_data.count())
    tt = time() - t0
    print("Prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(test_accuracy,4)))

    Prediction made in 86.879 seconds. Testing accuracy is 0.8734

In \[15\]:

    from pyspark.mllib.regression import LabeledPoint
    from numpy import array

    def attack_split(line):
        lines_split = line.split(",")
        numerical_line_split = [6,7,8,9,10,11,12,16,17,18,19,20,21,22,23,24,25,26,27,30,31,32,33,34,36,43,44,45,46,40,41,42,48,35]
        attack = 10.0
        if lines_split[47]== '':
            attack = 0.0
        elif lines_split[47] == 'Exploits':
            attack = 1.0
        elif lines_split[47] == 'Reconnaissance':
            attack = 2.0
        elif lines_split[47] == 'DoS':
            attack = 3.0
        elif lines_split[47] == 'Backdoor':
            attack = 4.0
        elif lines_split[47] == 'Fuzzers':
            attack = 5.0
        elif lines_split[47] == 'Worms':
            attack = 6.0
        elif lines_split[47] == 'Generic':
            attack = 7.0
        elif lines_split[47] == 'Analysis':
            attack = 8.0
        elif lines_split[47] == 'Shellcode':
            attack = 9.0
        return LabeledPoint(attack, array([float(x) for x in numerical_line_split]))

    category_data = unsw_rdd.map(attack_split)

In \[17\]:

    [training_data, testing_data] = category_data.randomSplit([0.7, 0.3], seed = 1234)

In \[18\]:

    from pyspark.mllib.classification import LogisticRegressionWithLBFGS
    from time import time


    t = time()
    logistic_model = LogisticRegressionWithLBFGS.train(train_data)
    diff = time() - t # to know how long it takes to complete Regression analysis

    print ("Classifier trained in {} seconds".format(round(diff,3)))

    Classifier trained in 62.467 seconds

In \[19\]:

    labels_prediction_train = training_data.map(lambda a: (abs(a.label)-abs(logistic_model.predict(a.features))))

In \[22\]:

    labels_prediction_train.take(12)

Out\[22\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[23\]:

    #training accuracy for normal
    t = time()
    train_accuracy_normal = labels_prediction_train.filter(lambda a: a==0).count() / float(training_data.count())
    diff = time() - t
    print ("Normal data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_normal,4)))

    Normal data prediction made in 101.107 seconds. Training accuracy is 0.8736

In \[24\]:

    #training accuracy for exploits
    t = time()
    train_accuracy_exploits = labels_prediction_train.filter(lambda a: a==1.0).count() / float(training_data.count())
    diff = time() - t
    print ("Exploits data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_exploits,4)))

    Exploits data prediction made in 108.727 seconds. Training accuracy is 0.0176

In \[25\]:

    #training accuracy for Reconnaissance
    t = time()
    train_accuracy_rec = labels_prediction_train.filter(lambda a: a==2.0).count() / float(training_data.count())
    diff = time() - t
    print ("Reconnaissance data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_rec,4)))

    Reconnaissance data prediction made in 106.451 seconds. Training accuracy is 0.0007

In \[26\]:

    #training accuracy for DoS
    t = time()
    train_accuracy_dos = labels_prediction_train.filter(lambda a: a==3.0).count() / float(training_data.count())
    diff = time() - t
    print ("DoS data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_dos,4)))

    DoS data prediction made in 104.507 seconds. Training accuracy is 0.0065

In \[28\]:

    #training accuracy for Backdoor
    t = time()
    train_accuracy_back = labels_prediction_train.filter(lambda a: a==4.0).count() / float(training_data.count())
    diff = time() - t
    print ("Backdoor data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_back,4)))

    Backdoor data prediction made in 118.853 seconds. Training accuracy is 0.0007

In \[29\]:

    #training accuracy for Fuzzers
    t = time()
    train_accuracy_fuzz = labels_prediction_train.filter(lambda a: a==5.0).count() / float(training_data.count())
    diff = time() - t
    print ("Fuzzers data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_fuzz,4)))

    Fuzzers data prediction made in 111.311 seconds. Training accuracy is 0.002

In \[30\]:

    #training accuracy for Worms
    t = time()
    train_accuracy_worm = labels_prediction_train.filter(lambda a: a==6.0).count() / float(training_data.count())
    diff = time() - t
    print ("Worms data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_worm,4)))

    Worms data prediction made in 107.17 seconds. Training accuracy is 0.0001

In \[31\]:

    #training accuracy for Generic
    t = time()
    train_accuracy_gen = labels_prediction_train.filter(lambda a: a==7.0).count() / float(training_data.count())
    diff = time() - t
    print ("Generic data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_gen,4)))

    Generic data prediction made in 104.546 seconds. Training accuracy is 0.0847

In \[32\]:

    #training accuracy for Analysis
    t = time()
    train_accuracy_any = labels_prediction_train.filter(lambda a: a==8.0).count() / float(training_data.count())
    diff = time() - t
    print ("Analysis data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_any,4)))

    Analysis data prediction made in 106.432 seconds. Training accuracy is 0.0011

In \[33\]:

    #training accuracy for Shellcode
    t = time()
    train_accuracy_shc = labels_prediction_train.filter(lambda a: a==9.0).count() / float(training_data.count())
    diff = time() - t
    print ("Shellcode data prediction made in {} seconds. Training accuracy is {}".format(round(diff,3), round(train_accuracy_shc,4)))

    Shellcode data prediction made in 108.152 seconds. Training accuracy is 0.0001

In \[34\]:

    labels_preds_testing = testing_data.map(lambda a: (abs(a.label)-abs(logistic_model.predict(a.features))))

In \[35\]:

    labels_preds_testing.take(12)

Out\[35\]:

    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In \[37\]:

    # testing accuracy for normal
    t = time()
    testing_accuracy_normal = labels_preds_testing.filter(lambda a: a==0.0).count() / float(testing_data.count())
    diff = time() - t
    print("Normal prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_normal,4)))

    Normal prediction made in 86.879 seconds. Testing accuracy is 0.8734

In \[38\]:

    # testing accuracy for Exploits
    t = time()
    testing_accuracy_exp = labels_preds_testing.filter(lambda a: a==1.0).count() / float(testing_data.count())
    diff = time() - t
    print("Exploits prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_exp,4)))

    Exploits prediction made in 86.879 seconds. Testing accuracy is 0.0174

In \[39\]:

    # testing accuracy for Reconnaisance
    t = time()
    testing_accuracy_rec = labels_preds_testing.filter(lambda a: a==2.0).count() / float(testing_data.count())
    diff = time() - t
    print("Reconnaisance prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_rec,4)))

    Reconnaisance prediction made in 86.879 seconds. Testing accuracy is 0.0007

In \[40\]:

    # testing accuracy for DoS
    t = time()
    testing_accuracy_dos = labels_preds_testing.filter(lambda a: a==3.0).count() / float(testing_data.count())
    diff = time() - t
    print("DoS prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_dos,4)))

    DoS prediction made in 86.879 seconds. Testing accuracy is 0.0063

In \[41\]:

    # testing accuracy for Backdoor
    t = time()
    testing_accuracy_back = labels_preds_testing.filter(lambda a: a==4.0).count() / float(testing_data.count())
    diff = time() - t
    print("Backdoor prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_back,4)))

    Backdoor prediction made in 86.879 seconds. Testing accuracy is 0.0007

In \[42\]:

    # testing accuracy for Fuzzers
    t = time()
    testing_accuracy_fuzz = labels_preds_testing.filter(lambda a: a==5.0).count() / float(testing_data.count())
    diff = time() - t
    print("Fuzzers prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_fuzz,4)))

    Fuzzers prediction made in 86.879 seconds. Testing accuracy is 0.0021

In \[43\]:

    # testing accuracy for Worms
    t = time()
    testing_accuracy_worm = labels_preds_testing.filter(lambda a: a==6.0).count() / float(testing_data.count())
    diff = time() - t
    print("Worms prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_worm,4)))

    Worms prediction made in 86.879 seconds. Testing accuracy is 0.0001

In \[44\]:

    # testing accuracy for Generic
    t = time()
    testing_accuracy_gen = labels_preds_testing.filter(lambda a: a==7.0).count() / float(testing_data.count())
    diff = time() - t
    print("Generic prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_gen,4)))

    Generic prediction made in 86.879 seconds. Testing accuracy is 0.0851

In \[45\]:

    # testing accuracy for Analysis
    t = time()
    testing_accuracy_any = labels_preds_testing.filter(lambda a: a==8.0).count() / float(testing_data.count())
    diff = time() - t
    print("Analysis prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_any,4)))

    Analysis prediction made in 86.879 seconds. Testing accuracy is 0.001

In \[46\]:

    # testing accuracy for Shellcode
    t = time()
    testing_accuracy_shc = labels_preds_testing.filter(lambda a: a==9.0).count() / float(testing_data.count())
    diff = time() - t
    print("Shellcode prediction made in {} seconds. Testing accuracy is {}".format(round(tt,3), round(testing_accuracy_shc,4)))

    Shellcode prediction made in 86.879 seconds. Testing accuracy is 0.0001

In \[60\]:

    print("Summary of accuracy for different attacks for both training and testing data: ")
    print("Training accuracy for Normal data: {}%  Testing accuracy for Normal data: {}%".format(round(train_accuracy_normal*100,3), round(testing_accuracy_normal*100,3)))
    print("Training accuracy for Exploits data: {}%  Testing accuracy for Exploits data: {}%".format(round(train_accuracy_exploits*100,3), round(testing_accuracy_exp*100,3)))
    print("Training accuracy for Reconnaisance data: {}%  Testing accuracy for Reconnaisance data: {}%".format(round(train_accuracy_rec*100,3), round(testing_accuracy_rec*100,3)))
    print("Training accuracy for DoS data: {}%  Testing accuracy for DoS data: {}%".format(round(train_accuracy_dos*100,3), round(testing_accuracy_dos*100,3)))
    print("Training accuracy for Backdoor data: {}%  Testing accuracy for Backdoor data: {}%".format(round(train_accuracy_back*100,3), round(testing_accuracy_back*100,3)))
    print("Training accuracy for Fuzzers data: {}%  Testing accuracy for Fuzzers data: {}%".format(round(train_accuracy_fuzz*100,3), round(testing_accuracy_fuzz*100,3)))
    print("Training accuracy for Worms data: {}%  Testing accuracy for Worms data: {}%".format(round(train_accuracy_worm*100,3), round(testing_accuracy_worm*100,3)))
    print("Training accuracy for Generic data: {}%  Testing accuracy for Generic data: {}%".format(round(train_accuracy_gen*100,3), round(testing_accuracy_gen*100,3)))
    print("Training accuracy for Analysis data: {}%  Testing accuracy for Analysis data: {}%".format(round(train_accuracy_any*100,3), round(testing_accuracy_any*100,3)))
    print("Training accuracy for Shellcode data: {}%  Testing accuracy for Shellcode data: {}%".format(round(train_accuracy_shc*100,3), round(testing_accuracy_shc*100,3)))

    Summary of accuracy for different attacks for both training and testing data: 
    Training accuracy for Normal data: 87.355%  Testing accuracy for Normal data: 87.337%
    Training accuracy for Exploits data: 1.757%  Testing accuracy for Exploits data: 1.744%
    Training accuracy for Reconnaisance data: 0.069%  Testing accuracy for Reconnaisance data: 0.07%
    Training accuracy for DoS data: 0.649%  Testing accuracy for DoS data: 0.633%
    Training accuracy for Backdoor data: 0.071%  Testing accuracy for Backdoor data: 0.07%
    Training accuracy for Fuzzers data: 0.195%  Testing accuracy for Fuzzers data: 0.207%
    Training accuracy for Worms data: 0.007%  Testing accuracy for Worms data: 0.006%
    Training accuracy for Generic data: 8.472%  Testing accuracy for Generic data: 8.513%
    Training accuracy for Analysis data: 0.107%  Testing accuracy for Analysis data: 0.101%
    Training accuracy for Shellcode data: 0.008%  Testing accuracy for Shellcode data: 0.01%

In \[ \]: